In this notebook we average the data accumulated over two runs for each GPT model for each type of experiment. 

Each run consisted of 100 experiments for each red teaming context. 

We average the metric results for each context and we calculate their standard deviations. 

In [ ]:
import numpy as np
import json
import os

In [ ]:
folders_arithmetic_low_35 = ['arithmetic_low_gpt_3.5_part_1', 'arithmetic_low_gpt_3.5_part_2']
folders_arithmetic_high_35 = ['arithmetic_high_gpt_3.5_part_1', 'arithmetic_high_gpt_3.5_part_2']
folders_puzzle_35 = ['puzzle_low_gpt_3.5_part_1', 'puzzle_low_gpt_3.5_part_2']

folders_arithmetic_low_4 = ['arithmetic_low_gpt_4_part_1', 'arithmetic_low_gpt_4_part_2']
folders_arithmetic_high_4 = ['arithmetic_high_gpt_4_part_1', 'arithmetic_high_gpt_4_part_2']
folders_puzzle_4 = ['puzzle_low_gpt_4_part_1', 'puzzle_low_gpt_4_part_2']

In [ ]:
folders = [
    folders_arithmetic_low_35, folders_arithmetic_high_35,
    folders_puzzle_35, folders_arithmetic_low_4,
    folders_arithmetic_high_4, folders_puzzle_4
]

In [ ]:
out_names = [
    'arithmetic_low_gpt_3.5_avg', 'arithmetic_high_gpt_3.5_avg',
    'puzzle_gpt_3.5_avg', 'arithmetic_low_gpt_4_avg',
    'arithmetic_high_gpt_4_avg', 'puzzle_gpt_4_avg'
]

In [ ]:
def average_metrics(a, b, a_rate, b_rate):
    n_runs = 100
    n = n_runs * a_rate
    m = n_runs * b_rate
    obj = {}
    metrics = ['abs_edit_distance', 'rel_edit_distance', 
               'abs_distance', 'rel_distance',
               'accuracy']
        
    for metric in metrics:
        a_mean = a[metric]['mean']
        b_mean = b[metric]['mean']
        
        a_std = a[metric]['std']
        b_std = b[metric]['std']
        n = 100
        combined_var = ((n * (a_std**2) + m * (b_std**2)) / (n + m)) + \
                  (n * m * ((a_mean - b_mean) ** 2) / ((n + m) ** 2))
        
        obj[metric] = {
            'mean': (a_mean * n + b_mean * m) / (n+m),
            'std': np.sqrt(combined_var)
        }
    
    return obj

In [ ]:
for files, out in zip(folders, out_names):
    with open(f'{files[0]}/results.json', 'r') as f:
        file_0 = json.loads(f.read())
    with open(f'{files[1]}/results.json', 'r') as f:
        file_1 = json.loads(f.read())
    
    out_arr = []
    
    for a, b in zip(file_0, file_1):
        out_obj = {
            'name': a['name'],
            'no_context': average_metrics(a['no_context'], b['no_context'], a['extraction_rate'], b['extraction_rate']),
            'context': average_metrics(a['context'], b['context'], a['extraction_rate'], b['extraction_rate']),
            'extraction_rate': np.mean([a['extraction_rate'], b['extraction_rate']])
        }
        out_arr.append(out_obj)
        
    with open(f'{out}_results_new.json', 'w') as f:
        f.write(json.dumps(out_arr, indent=4))